<a href="https://colab.research.google.com/github/mehedihasan-cse/LLM/blob/main/Response_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install sentence-transformers

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util

# Load the pre-trained model
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
sentence_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')


df = pd.read_csv('/content/Demo Data 1 - Sheet1.csv')

df.head()



def get_similarity(reference_answers, student_answer):
    # Encode sentences to get their embeddings
    ref_embeddings = sentence_model.encode(reference_answers, convert_to_tensor=True)
    student_embedding = sentence_model.encode(student_answer, convert_to_tensor=True)

    # Compute cosine similarities
    cosine_scores = util.pytorch_cos_sim(student_embedding, ref_embeddings)

    # Calculate the average similarity score
    average_score = torch.mean(cosine_scores).item()
    return average_score

def get_mark(average_score):
    # Convert similarity score to a mark out of 5
    if average_score >= 0.9:
        return 5
    elif average_score >= 0.8:
        return 4
    elif average_score >= 0.7:
        return 3
    elif average_score >= 0.6:
        return 2
    elif average_score >= 0.5:
        return 1
    else:
        return 0

# Evaluate student answers
df['similarity_score'] = df.apply(lambda row: get_similarity([row['answer_1'], row['answer_2'], row['answer_3'], row['answer_4']], row['student_answer']), axis=1)
df['mark'] = df['similarity_score'].apply(get_mark)

# Display results
print(df)